# Model and data transfer between servers

# Storage space on the GPU server

Storage space is limited on the GPU server, so please do not store massive amounts of images in your home folder. The angle90 images are already on there (in /datb/BotsNLE/angle90), so you can simply use that path of create a link (ln -s) to that path to the folder you work in. That way we all use the same files and that saves storage space.

If you do wish to use several GB of storage space, please contact Jeroen to facilitate that (we have more storage space on other disks).

# Transferring folders from bot to server

You can use rsync to transfer an entire folder from the bot to the gpu server. You should use rsync from a Terminal, because you will have to type in your password. For example:

`rsync -avr my_images jeroen@gpuserver.hhs.nl:notebooks`

will transfer all in the folder `my_images` to the notebooks folder of the user jeroen on the GPU server. You will have to type in the users password to do this. Instead of jeroen, use your own username.

# Saving a learned model

The RPi is too slow for training, validation and optimization of model, use the GPU server for that (or your laptop). Once the model is trained you will want to transfer that to the Raspberry Pi to evaluate. The following example assumes that the variable `model` 

In [ ]:
import torch
from torch  import nn
import matplotlib.pyplot as plt

In [ ]:
# This is an untrained dummy model, just to show how saving and loading of weights work

model = nn.Sequential(
    nn.Linear(15*40,3)
)

In [ ]:
# training, tuning, validation code

Once your model is ready, use the following steps:
1. transfer the model to CPU
1. grab the learned weights
1. save the weights to file
1. transfer the file with the weight to the Bot
1. On the bot, use the exact same code to construct a model with the same structure
1. Load the trained parameters from teh file
1. insert the trained parameters into the model 

In [ ]:
model.cpu()    #1

In [ ]:
weights = model.state_dict()  #2

In [ ]:
torch.save(weights, 'model_weights') #3

# #4

To transfer the weights file to the bot, you will have do that from the bot. The bot can see the gpu server and not the other way around. You will need to do this from a Terminal, because you will need to type in your password.

scp jeroen@gpuserver.hhs.nl:notebooks/ds3/model_weights .

And af course, you need to use your username instead of 'jeroen' and use the path and filename that corresponds to where the file is. You can replace the . with a foldername to transfer the weights file to that folder.

Then on the bot:

In [ ]:
# This is an untrained dummy model, just to show how saving and loading of weights work

model = nn.Sequential(    #5
    nn.Linear(15*40,3)
)

In [ ]:
weights = torch.load('model_weights')   #6

Note that this only works if the model has the exact same names and structure.

In [ ]:
model.load_state_dict(weights)    #7

# Inferencing on images

Now that you have transfered your model to the model, you will use that on images from the camera. To do this, it is important to prepare the images in the exact same way as you did when validating the images. For example, if you turned the images into grayscale, cropped off the top part and resized the images, you should use the same transformation here. You typically do NOT use any augmentations you have used on the trainingset to learn a more robust model (like random crops, rotations and flips).

One other important thing is that you must match the dimensionality of the data that was used for training.

In [ ]:
%run car.ipynb  # this will not run on the gpu server, and you can also use your own code

In [ ]:
from PIL import Image
from torchvision import transforms

In [ ]:
c = Car.get()

In [ ]:
image_array = c.new_image()    # grab an image

In [ ]:
image_pil = Image.fromarray(image_array)

Because you cannot run the car-code on the gpu server, the code below will load an image from a file (the server has no camera) to demonstrate how image preparation for inferencing works.

In [ ]:
image_pil = Image.open('angle90/0000-0.02--0.5625.jpg')

In [ ]:
transformations = transforms.Compose([   
    transforms.Grayscale(),
    transforms.Resize(30),
    transforms.ToTensor()
])

When you check the shape of your image tensor, it will probably be (1,30,40) if the aspect ratio is preserved. The numbers can vary (e.g. if you do not use Grayscale or use other numbers for resize).

In [ ]:
image_tensor = transformations(image_pil)

In [ ]:
plt.imshow(image_tensor[0])

In [ ]:
image_tensor.shape

You could do cropping inside or outside of the transformation. Inside the transformation, you have to use transforms.Lamba() with a function in which you crop the image (or write a cutsom transformation class). Outside the transformation, you can simply slice the tensor:

In [ ]:
image_tensor = image_tensor[:,15:]

In [ ]:
plt.imshow(image_tensor[0])

In [ ]:
image_tensor.shape

Now you have to match the dimensionality of the training data. In training, the input tensor consists of multiple images and the first dimension indicates the image number. Here we can add that dimension with an unsqueeze:

In [ ]:
X = image_tensor.unsqueeze(0)

In [ ]:
X.shape

And now your image is ready to use in the model

In [ ]:
prediction = model(X)